In [102]:
import sys
print(sys.executable)

c:\Users\Beeclick\Documents\MIAAD\S3\Data Mining\Project\data-mining-project\DataMiningEnv\Scripts\python.exe


### Import des bibliothèques

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Chargement des données brutes

In [104]:
clients = pd.read_csv("../data/raw/Clients.csv")
catalogue = pd.read_csv("../data/raw/Catalogue.csv")
immatriculations = pd.read_csv("../data/raw/immatriculations_small.csv")
marketing = pd.read_csv("../data/raw/Marketing.csv")

# Nettoyage des données pour les clients

In [105]:
# Affichage des premières lignes
print("Aperçu des données brutes:")
clients.head()

Aperçu des données brutes:


,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2eme voiture,immatriculation
0,25,F,159,En Couple,2,false,3467 SB 72
1,53,M,594,En Couple,2,false,113 LY 42
2,20,F,949,En Couple,1,false,925 WK 87
3,29,M,571,En Couple,2,false,3279 RV 81
4,47,M,502,En Couple,1,false,82 RZ 54


In [106]:
# Remplacer les valeurs incorrectes (erreurs de saisie) par NaN
clients.replace(['?', 'N/D', ' '], pd.NA, inplace=True)

In [107]:
# Vérification des types de données et des valeurs manquantes
print("\nInformations sur les données:")
clients.info()
print("\nValeurs manquantes par colonne:")
clients.isnull().sum()


Informations sur les données:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   age                 99807 non-null   object
 1   sexe                99709 non-null   object
 2   taux                99806 non-null   object
 3   situationFamiliale  99681 non-null   object
 4   nbEnfantsAcharge    99800 non-null   object
 5   2eme voiture        99780 non-null   object
 6   immatriculation     100000 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB

Valeurs manquantes par colonne:


age                   193
sexe                  291
taux                  194
situationFamiliale    319
nbEnfantsAcharge      200
2eme voiture          220
immatriculation         0
dtype: int64

In [108]:
# Conversion en type numérique en remplaçant les valeurs non convertibles par NaN
clients['age'] =  pd.to_numeric(clients['age'], errors='coerce')
clients['taux'] = pd.to_numeric(clients['taux'], errors='coerce')
clients['nbEnfantsAcharge'] = pd.to_numeric(clients['nbEnfantsAcharge'], errors='coerce')

In [109]:
# Recherche des doublons
duplicates = clients.duplicated().sum()
print(f"\nNombre de doublons : {duplicates}")


Nombre de doublons : 0


In [110]:
# Suppression des doublons
# df = df.drop_duplicates()

In [111]:
# Compter les occurrences pour chaque colonne de type 'object'
for col in clients.columns:
    if clients[col].dtype == 'object':
        print(clients[col].value_counts())
        print()

# or
# clients['sexe'].unique() #...

sexe
M           68283
F           29388
Homme         743
Masculin      684
Féminin       322
Femme         289
Name: count, dtype: int64

situationFamiliale
En Couple      63855
Célibataire    29841
Seule           4989
Marié(e)         664
Seul             282
Divorcée          50
Name: count, dtype: int64

2eme voiture
false    86845
true     12935
Name: count, dtype: int64

immatriculation
835 JQ 59     2
4229 XL 63    2
7084 RU 46    2
3648 SM 17    2
3142 RV 53    2
             ..
5659 SO 55    1
6760 YM 11    1
385 UN 57     1
4056 HI 13    1
3467 SB 72    1
Name: count, Length: 99989, dtype: int64



In [112]:
# Remplacer les valeurs dans la colonne "sexe"
clients['sexe'] = clients['sexe'].replace({
    'Homme': 'M',
    'Masculin': 'M',
    'Féminin': 'F',
    'Femme': 'F',
})

In [113]:
# Remplacer les valeurs dans la colonne "sexe"
clients['situationFamiliale'] = clients['situationFamiliale'].replace({
    'Seule': 'Célibataire',
    'Seul': 'Célibataire',
})

In [114]:
clients.describe(include='object')

,sexe,situationFamiliale,2eme voiture,immatriculation
count,99709,99681,99780,100000
unique,2,4,2,99989
top,M,En Couple,false,835 JQ 59
freq,69710,63855,86845,2


In [115]:
clients.describe()

,age,taux,nbEnfantsAcharge
count,99807.000000,99806.000000,99800.000000
mean,43.733335,607.800503,1.245150
std,18.355480,336.279121,1.388624
min,-1.000000,-1.000000,-1.000000
25%,28.000000,420.000000,0.000000
50%,42.000000,520.000000,1.000000
75%,57.000000,827.000000,2.000000
max,84.000000,1399.000000,4.000000


### Gérer les valeurs manquantes

In [116]:
# Création d'un nouveau dataframe pour Imputation et Suppression
clients_imputed = clients.copy()
clients_dropped = clients.copy()

#### A-Remplacez les valeurs manquantes par mean or mode or median

In [117]:
# Remplacement des valeurs manquantes pour les colonnes numériques par la médiane
clients_imputed['age'] = clients_imputed['age'].fillna(clients_imputed['age'].median())
clients_imputed['taux'] = clients_imputed['taux'].fillna(clients_imputed['taux'].median())
clients_imputed['nbEnfantsAcharge'] = clients_imputed['nbEnfantsAcharge'].fillna(clients_imputed['nbEnfantsAcharge'].median())

In [118]:
# Remplacement des valeurs manquantes pour les colonnes catégorielles par la valeur la plus fréquente
clients_imputed['sexe'] = clients_imputed['sexe'].fillna(clients_imputed['sexe'].mode()[0])
clients_imputed['situationFamiliale'] = clients_imputed['situationFamiliale'].fillna(clients_imputed['situationFamiliale'].mode()[0])
clients_imputed['2eme voiture'] = clients_imputed['2eme voiture'].fillna(clients_imputed['2eme voiture'].mode()[0])

In [119]:
clients_imputed.describe(include='object')

,sexe,situationFamiliale,2eme voiture,immatriculation
count,100000,100000,100000,100000
unique,2,4,2,99989
top,M,En Couple,false,835 JQ 59
freq,70001,64174,87065,2


In [120]:
clients_imputed.describe()

,age,taux,nbEnfantsAcharge
count,100000.000000,100000.000000,100000.000000
mean,43.729990,607.630170,1.244660
std,18.337916,335.974983,1.387278
min,-1.000000,-1.000000,-1.000000
25%,28.000000,420.000000,0.000000
50%,42.000000,520.000000,1.000000
75%,57.000000,826.000000,2.000000
max,84.000000,1399.000000,4.000000


#### B-Supprimer les lignes avec des valeurs manquantes

In [121]:
# Pourcentage de valeurs manquantes
print(clients.isnull().sum() / len(clients) * 100)  

age                   0.193
sexe                  0.291
taux                  0.194
situationFamiliale    0.319
nbEnfantsAcharge      0.200
2eme voiture          0.220
immatriculation       0.000
dtype: float64


In [122]:
# Suppression des lignes avec des valeurs manquantes
clients_dropped = clients_dropped.dropna()

# Vérification des valeurs manquantes dans le nouveau dataframe
print(clients_dropped.isnull().sum())


age                   0
sexe                  0
taux                  0
situationFamiliale    0
nbEnfantsAcharge      0
2eme voiture          0
immatriculation       0
dtype: int64


### Sauvegarde des données nettoyées

In [123]:
# J'utilise le dataframe clients_imputed pour le premier essai
clients_imputed.to_csv('../data/processed/clients_cleaned.csv', index=False)
print("\nDonnées nettoyées sauvegardées dans 'clients_cleaned.csv'")


Données nettoyées sauvegardées dans 'clients_cleaned.csv'


# Nettoyage des données pour Catalogue

In [124]:
# Affichage des premières lignes
print("Aperçu des données brutes:")
catalogue.head()

Aperçu des données brutes:


,marque,nom,puissance,longueur,nbPlaces,nbPortes,couleur,occasion,prix
0,Volvo,S80 T6,272,très longue,5,5,blanc,False,50500
1,Volvo,S80 T6,272,très longue,5,5,noir,False,50500
2,Volvo,S80 T6,272,très longue,5,5,rouge,False,50500
3,Volvo,S80 T6,272,très longue,5,5,gris,True,35350
4,Volvo,S80 T6,272,très longue,5,5,bleu,True,35350


In [125]:
# Remplacer les valeurs incorrectes (erreurs de saisie) par NaN
catalogue.replace(['?', 'N/D', ' '], pd.NA, inplace=True)

In [126]:
# Vérification des types de données et des valeurs manquantes
print("\nInformations sur les données:")
catalogue.info()
print("\nValeurs manquantes par colonne:")
catalogue.isnull().sum()


Informations sur les données:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   marque     270 non-null    object
 1   nom        270 non-null    object
 2   puissance  270 non-null    int64 
 3   longueur   270 non-null    object
 4   nbPlaces   270 non-null    int64 
 5   nbPortes   270 non-null    int64 
 6   couleur    270 non-null    object
 7   occasion   270 non-null    bool  
 8   prix       270 non-null    int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 17.3+ KB

Valeurs manquantes par colonne:


marque       0
nom          0
puissance    0
longueur     0
nbPlaces     0
nbPortes     0
couleur      0
occasion     0
prix         0
dtype: int64

In [127]:
# Recherche des doublons
duplicates = catalogue.duplicated().sum()
print(f"\nNombre de doublons : {duplicates}")


Nombre de doublons : 0


In [128]:
# Compter les occurrences pour chaque colonne de type 'object'
for col in catalogue.columns:
    if catalogue[col].dtype == 'object':
        print(catalogue[col].value_counts())
        print()

marque
Volkswagen    40
Renault       40
BMW           20
Mercedes      20
Audi          20
Nissan        15
Jaguar        10
Saab          10
Volvo         10
Fiat          10
Lancia        10
Mini          10
Peugeot       10
Ford          10
Skoda          5
Seat           5
Kia            5
Hyundaï        5
Honda          5
Dacia          5
Daihatsu       5
Name: count, dtype: int64

nom
S80 T6              10
Touran 2.0 FSI      10
Polo 1.2 6V         10
New Beatle 1.8      10
Golf 2.0 FSI        10
9.3 1.8T            10
Megane 2.0 16V      10
Vel Satis 3.5 V6    10
1007 1.4            10
Espace 2.0T         10
Laguna 2.0T         10
X-Type 2.5 V6       10
Ypsilon 1.4 16V     10
A200                10
S500                10
Copper 1.6 16V      10
A3 2.0 FSI          10
A2 1.4              10
M5                  10
Mondeo 1.8          10
Croma 2.2           10
120i                10
Superb 2.8 V6        5
Toledo 1.6           5
Almera 1.8           5
Maxima 3.0 V6        5
Primera

In [129]:
catalogue.describe(include='object')

,marque,nom,longueur,couleur
count,270,270,270,270
unique,21,32,4,5
top,Volkswagen,S80 T6,longue,blanc
freq,40,10,90,54


In [130]:
catalogue.describe()

,puissance,nbPlaces,nbPortes,prix
count,270.000000,270.000000,270.000000,270.000000
mean,157.592593,5.222222,4.814815,26668.055556
std,90.551289,0.629707,0.580798,19050.121112
min,55.000000,5.000000,3.000000,7500.000000
25%,109.000000,5.000000,5.000000,16029.000000
50%,147.000000,5.000000,5.000000,20597.500000
75%,170.000000,5.000000,5.000000,30000.000000
max,507.000000,7.000000,5.000000,101300.000000


### Sauvegarde des données nettoyées

In [131]:
catalogue.to_csv('../data/processed/catalogue_cleaned.csv', index=False)
print("\nDonnées nettoyées sauvegardées dans 'catalogue_cleaned.csv'")


Données nettoyées sauvegardées dans 'catalogue_cleaned.csv'


# Nettoyage des données pour Immatriculations

In [132]:
# Affichage des premières lignes
print("Aperçu des données brutes:")
immatriculations.head()

Aperçu des données brutes:


,immatriculation,marque,nom,puissance,longueur,nbPlaces,nbPortes,couleur,occasion,prix
0,3467 SB 72,Seat,Toledo 1.6,102,longue,5,5,bleu,False,18880
1,113 LY 42,Renault,Vel Satis 3.5 V6,245,très longue,5,5,noir,False,49200
2,925 WK 87,Jaguar,X-Type 2.5 V6,197,longue,5,5,blanc,False,37100
3,3279 RV 81,Renault,Laguna 2.0T,170,longue,5,5,gris,False,27300
4,82 RZ 54,Fiat,Croma 2.2,147,longue,5,5,bleu,False,24780


In [133]:
# Remplacer les valeurs incorrectes (erreurs de saisie) par NaN
immatriculations.replace(['?', 'N/D', ' '], pd.NA, inplace=True)

In [134]:
# Vérification des types de données et des valeurs manquantes
print("\nInformations sur les données:")
immatriculations.info()
print("\nValeurs manquantes par colonne:")
immatriculations.isnull().sum()


Informations sur les données:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   immatriculation  99989 non-null  object
 1   marque           99989 non-null  object
 2   nom              99989 non-null  object
 3   puissance        99989 non-null  int64 
 4   longueur         99989 non-null  object
 5   nbPlaces         99989 non-null  int64 
 6   nbPortes         99989 non-null  int64 
 7   couleur          99989 non-null  object
 8   occasion         99989 non-null  bool  
 9   prix             99989 non-null  int64 
dtypes: bool(1), int64(4), object(5)
memory usage: 7.0+ MB

Valeurs manquantes par colonne:


immatriculation    0
marque             0
nom                0
puissance          0
longueur           0
nbPlaces           0
nbPortes           0
couleur            0
occasion           0
prix               0
dtype: int64

In [135]:
# Compter les occurrences pour chaque colonne de type 'object'
for col in immatriculations.columns:
    if immatriculations[col].dtype == 'object':
        print(immatriculations[col].value_counts())
        print()

immatriculation
4656 EJ 74    1
3467 SB 72    1
113 LY 42     1
925 WK 87     1
3279 RV 81    1
             ..
9896 TW 50    1
6484 MS 45    1
3696 JS 92    1
9339 BW 87    1
8290 SD 41    1
Name: count, Length: 99989, dtype: int64

marque
Audi          14668
BMW           14529
Renault       12449
Jaguar         9418
Volkswagen     7867
Mercedes       7541
Volvo          6369
Peugeot        4852
Saab           4282
Fiat           3796
Ford           3767
Nissan         3378
Skoda          1634
Kia            1154
Daihatsu       1097
Seat           1094
Mini            811
Dacia           696
Lancia          587
Name: count, dtype: int64

nom
A2 1.4              14268
M5                  12660
X-Type 2.5 V6        9418
S80 T6               6369
Vel Satis 3.5 V6     6101
S500                 5256
Polo 1.2 6V          4883
1007 1.4             4852
9.3 1.8T             4282
Croma 2.2            3796
Laguna 2.0T          3786
Mondeo 1.8           3767
Golf 2.0 FSI         2634
Megane 2.0

In [136]:
immatriculations.describe()

,puissance,nbPlaces,nbPortes,prix
count,99989.000000,99989.0,99989.000000,99989.000000
mean,198.561272,5.0,4.868646,35730.097251
std,136.715032,0.0,0.495437,25769.478289
min,55.000000,5.0,3.000000,7500.000000
25%,75.000000,5.0,5.000000,18310.000000
50%,150.000000,5.0,5.000000,25970.000000
75%,245.000000,5.0,5.000000,49200.000000
max,507.000000,5.0,5.000000,101300.000000


In [137]:
immatriculations.describe(include='object')

,immatriculation,marque,nom,longueur,couleur
count,99989,99989,99989,99989,99989
unique,99989,19,28,4,5
top,4656 EJ 74,Audi,A2 1.4,très longue,bleu
freq,1,14668,14268,33667,20085


### Sauvegarde des données nettoyées

In [138]:
immatriculations.to_csv('../data/processed/immatriculations_cleaned.csv', index=False)
print("\nDonnées nettoyées sauvegardées dans 'immatriculations_cleaned.csv'")


Données nettoyées sauvegardées dans 'immatriculations_cleaned.csv'
